In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
# 법령 XML 상세조회 (시행기준. target=eflaw)

#기본 url
url_base = " http://www.law.go.kr/DRF/lawService.do"

#조회 url 세부 설정
user_id = 'bmsong' #open API ID (bmsong@kau.ac.kr의 아이디 부분)
target = "eflaw" #법령 조회시
MST = "219009" #ID 또는 MST #위험물안전관리법 MST 219009
#ID = "009502" 
            #ID 위험물안전관리법 법령ID 009502 (설명에는 ID로 조회하면 현행법령이 조회된다는데, 
            #target을 eflaw로 하면 ID로 요청해도, MST로 요청해도 같은 것 같다.
Type = "XML" #출력 형태 : HTML 또는 XML

url_sub = "?" +\
        "OC=" + user_id +\
        "&target=" + target +\
        "&MST=" + MST +\
        "&type=" + Type

#최종 url
url = url_base + url_sub

print(url)

response = requests.get(url)

#(1) response.text 사용하는 방법
#soup = BeautifulSoup(response.text.encode('utf-8'), 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)

#(2) response.content 사용하는 방법
soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)


# soup

 http://www.law.go.kr/DRF/lawService.do?OC=bmsong&target=eflaw&MST=219009&type=XML


In [3]:
lc_basic_df = pd.DataFrame() #DataFrame for Law Content 

data = soup.find('법령ID') #MST 정보
lc_basic_df['법령ID'] = [data.get_text()]

data = soup.find('법종구분') 
lc_basic_df['법종구분']= [data.get_text()]

data = soup.find('법령명_한글') 
lc_basic_df['법령명']= [data.get_text()]

data = soup.find('시행일자')
lc_basic_df['시행일자'] = [data.get_text()]

data = soup.find('공포일자') 
lc_basic_df['공포일자']= [data.get_text()]

data = soup.find('소관부처') 
lc_basic_df['소관부처']= [data.get_text()]

lc_basic_df

,법령ID,법종구분,법령명,시행일자,공포일자,소관부처
0,009502,법률,위험물안전관리법,20210610,20200609,소방청


In [11]:
# data = soup.find_all('조문단위')
# df_info = pd.DataFrame()
# info = []
# for i in data:
#     detail = {}
#     detail["index"] = np.nan
#     detail["조문제목"] = np.nan
#     detail["조문내용"] = np.nan
#     detail["항문내용"] = np.nan
#     detail["호문내용"] = np.nan
#     if i.find("조문여부").get_text() == "조문":
#         조문제목 = i.find("조문내용").get_text().strip().split(")")[0]+")"
#         detail["조문제목"] = 조문제목
#         if i.find("조문내용"):
#             조문내용 = i.find("조문내용").get_text().strip()[len(조문제목):-1].strip()
#             detail["조문내용"] = 조문내용
        
#         df_info = df_info.append(detail,ignore_index = True)
        
#         if i.find("항"):
#             for j in i.find_all("항"):
#                 if j.find("항내용"):
#                     detail["항문내용"] = j.항내용.get_text().strip()
#                     df_info = df_info.append(detail,ignore_index = True)
#                 if j.find("호내용"):
#                     for k in j.find_all("호내용"):
#                         detail["호문내용"] = k.get_text().strip()
#                         df_info = df_info.append(detail,ignore_index = True)
#                 detail["호문내용"] = np.nan
# df_info = df_info[["조문제목","조문내용","항문내용","호문내용"]]
# df_info

,조문제목,조문내용,항문내용,호문내용
0,제1조(목적),이 법은 위험물의 저장ㆍ취급 및 운반과 이에 따른 안전관리에 관한 사항을 규정함으로...,NaN,NaN
1,제2조(정의),,NaN,NaN
2,제2조(정의),,①이 법에서 사용하는 용어의 정의는 다음과 같다.,NaN
3,제2조(정의),,①이 법에서 사용하는 용어의 정의는 다음과 같다.,"1. ""위험물""이라 함은 인화성 또는 발화성 등의 성질을 가지는 것으로서 대통령령이..."
4,제2조(정의),,①이 법에서 사용하는 용어의 정의는 다음과 같다.,"2. ""지정수량""이라 함은 위험물의 종류별로 위험성을 고려하여 대통령령이 정하는 수..."
...,...,...,...,...
222,제39조(과태료),,③ 삭제 <2014.12.30>,NaN
223,제39조(과태료),,④ 삭제 <2014.12.30>,NaN
224,제39조(과태료),,⑤ 삭제 <2014.12.30>,NaN
225,제39조(과태료),,⑥제4조 및 제5조제2항 각 호 외의 부분 후단의 규정에 따른 조례에는 200만원 ...,NaN


In [4]:
data = soup.find_all('조문단위')
df_info = pd.DataFrame()
info = []
for i in data:
    info_str = ""
    if i.find("조문여부").get_text() == "조문":
        info_str += i.조문내용.get_text().strip()
        if i.항:
            hang_nums = i.find_all("항번호")
            ho_nums = i.find_all("호번호")
            for idx, p in enumerate(i.find_all("호내용")):
                info_str += p.get_text().split(ho_nums[idx].get_text())[1].strip()
            for hang_idx, k in enumerate(i.find_all("항내용")):
                info_str += k.get_text().strip().split(hang_nums[hang_idx].get_text())[1].strip()
                for ho_idx ,j in enumerate(i.find_all("호내용")):
                    info_str += j.get_text().strip().split(ho_nums[ho_idx].get_text())[1].strip()
        info.append(info_str)
df_info["조내용"] = info
df_info

,조내용
0,제1조(목적) 이 법은 위험물의 저장ㆍ취급 및 운반과 이에 따른 안전관리에 관한 사...
1,"제2조(정의)""위험물""이라 함은 인화성 또는 발화성 등의 성질을 가지는 것으로서 대..."
2,제3조(적용제외) 이 법은 항공기ㆍ선박(선박법 제1조의2제1항의 규정에 따른 선박을...
3,제3조의2(국가의 책무)위험물의 유통실태 분석위험물에 의한 사고 유형의 분석사고 예...
4,제4조(지정수량 미만인 위험물의 저장ㆍ취급) 지정수량 미만인 위험물의 저장 또는 취...
5,제5조(위험물의 저장 및 취급의 제한)시ㆍ도의 조례가 정하는 바에 따라 관할소방서장...
6,제6조(위험물시설의 설치 및 변경 등)주택의 난방시설(공동주택의 중앙난방시설을 제외...
7,제7조(군용위험물시설의 설치 및 변경에 대한 특례)군사목적 또는 군부대시설을 위한 ...
8,제8조(탱크안전성능검사)위험물을 저장 또는 취급하는 탱크로서 대통령령이 정하는 탱크...
9,제9조(완공검사)제6조제1항의 규정에 따른 허가를 받은 자가 제조소등의 설치를 마쳤...


In [5]:
df_info.to_excel('위험물안전관리법.xlsx',index = False)